## Calling libc

Or anything else that's linked into your process memory.

In [ ]:
ccall(:puts, Cint, (Cstring,), "Hello")

In [ ]:
@time ccall(:sleep, Cuint, (Cuint,), 3)

## Calling shared libraries

Let's compile a little C program as a shared library:

In [ ]:
;cat hello.c

In [ ]:
;gcc -shared -fPIC -o libhello.so hello.c

In [ ]:
ccall((:hello, "./libhello.so"), Cvoid, (Cstring,), "ORNL")

In [ ]:
ccall((:hello, "./libhello.so"), Cvoid, (Cstring,), "Greg")

In [ ]:
floatmax()

In [ ]:
ccall((:sqr, "./libhello.so"), Cdouble, (Cdouble,), sqrt(floatmax()))

## Callbacks: calling Julia from C

In [ ]:
T = Float64

# C-friendly callback function
function callback(p_a::Ptr{T}, p_b::Ptr{T})::Cint
    a = unsafe_load(p_a)
    b = unsafe_load(p_b)
    a < b ? -1 : b < a ? 1 : 0
end

In [ ]:
p = [1.2, 3.4]

In [ ]:
p_a = pointer(p, 1)
p_b = pointer(p, 2)
callback(p_b, p_a)

In [ ]:
# get C-callable function pointer
p_callback = @cfunction(callback, Cint, (Ptr{T}, Ptr{T}))

In [ ]:
A = randn(10)

In [ ]:
# call C's qsort function
ccall(:qsort, Cvoid,
    (Ptr{T}, Csize_t, Csize_t, Ptr{Cvoid}),
    A, length(A), sizeof(T), p_callback)

In [ ]:
A

In [ ]:
function qsort!((<)::Function, A::Vector{T}) where T

    # C-friendly callback function
    function callback(p_a::Ptr{T}, p_b::Ptr{T})::Cint
        a = unsafe_load(p_a)
        b = unsafe_load(p_b)
        a < b ? -1 : b < a ? 1 : 0
    end

    # get C-callable function pointer
    p_callback = @cfunction($callback, Cint, (Ptr{T}, Ptr{T}))

    # call C's qsort function
    ccall(:qsort, Cvoid,
        (Ptr{T}, Csize_t, Csize_t, Ptr{Cvoid}),
        A, length(A), sizeof(T), p_callback)
    
    return A
end

# default comparison by `isless` function
qsort!(A::Vector) = qsort!(isless, A)

In [ ]:
A = randn(10)

In [ ]:
qsort!(A)

In [ ]:
qsort!((x,y)->abs(y) < abs(x), A)

In [ ]:
absgt(x, y) = abs(y) < abs(x)

In [ ]:
qsort!(absgt, A)

In [ ]:
B = rand(-10:10, 20)

In [ ]:
qsort!(B)

In [ ]:
qsort!((x, y)->abs(y) < abs(x), B)

In [ ]:
qsort!(absgt, B)

## Embedding Julia in C/Fortran

See https://docs.julialang.org/en/latest/manual/embedding/